# Another attempt at MC Simulation on AHP/ANP

The ideas are the following:

1. There is a class MCAnp that has a sim() method that will simulate any Prioritizer
2. MCAnp also has a sim_fill() function that does fills in the data needed for a single simulation

## Import needed libs

In [1]:
import pandas as pd
import sys 
import os
sys.path.insert(0, os.path.abspath("../"))
import numpy as np
from scipy.stats import triang
from copy import deepcopy
from pyanp.priority import pri_eigen
from pyanp.pairwise import Pairwise
from pyanp.ahptree import AHPTree
from pyanp.direct import Direct

# MCAnp class

In [2]:
def ascale_mscale(val:(float,int))->float:
    if val is None:
        return 0
    elif val < 0:
        val = -val
        val += 1
        val = 1.0/val
        return val
    else:
        return val+1
    
def mscale_ascale(val:(float,int))->float:
    if val == 0:
        return None
    elif val >= 1:
        return val - 1
    else:
        val = 1/val
        val = val-1
        return -val

In [33]:
DEFAULT_DISTRIB = triang(c=0.5, loc=-1.5, scale=3.0)
def avote_random(avote):
    """
    Returns a random additive vote in the neighborhood of the additive vote avote
    according to the default disribution DEFAULT_DISTRIB
    """
    if avote is None:
        return None
    raw_val = DEFAULT_DISTRIB.rvs(size=1)[0]
    return avote+raw_val


def mvote_random(mvote):
    """
    Returns a random multiplicative vote in the neighborhhod of the multiplicative vote mvote
    according to the default distribution DEFAULT_DISTRIB.  This is handled by converting
    the multiplicative vote to an additive vote, calling avote_random() and converting the
    result back to an additive vote
    """
    avote = mscale_ascale(mvote)
    rval_a = avote_random(avote)
    rval = ascale_mscale(rval_a)
    return rval

def direct_random(direct, max_percent_chg=0.2)->float:
    """
    Returns a random direct data value near the value `direct'.  This function
    creates a random percent change, between -max_percent_chg and +max_percent_chg, and
    then changes the direct value by that factor, and returns it.
    """
    pchg = np.random.uniform(low=-max_percent_chg, high=max_percent_chg)
    return direct * (1 + pchg)
    
class MCAnp:
    def __init__(self):
        # Setup the random pairwise vote generator
        self.pwvote_random = mvote_random
        # Setup the random direct vote generator
        self.directvote_random = direct_random
        # Set the default user to use across the simulation
        # follows the standard from Pairwise class, i.e. it can be a list
        # of usernames, a single username, or None (which means total group average)
        self.username = None
        # What is the pairwise priority calculation?
        self.pwprioritycalc = pri_eigen
        
    def sim_fill(self, src, dest):
        """
        Fills in data on a structure prior to doing the simulation calculations.
        This function calls sim_NAME_fill depending on the class of the src object.
        If the dest object is None, we create a dest object by calling deepcopy().
        In either case, we always return the allocated dest object
        """
        if dest is None:
            dest = deepcopy(src)
        # Which kind of src do we have
        if isinstance(src, np.ndarray):
            # We are simulating on a pairwise comparison matrix
            return sim_pwmat_fill(src, dest)
        elif isinstance(src, Pairwise):
            # We are simulating on a multi-user pairwise comparison object
            return sim_pw_fill(src, dest)
        elif isinstance(src, AHPTree):
            # We are simulating on an ahp tree object
            return sim_ahptree_fill(src, dest)
        else:
            raise ValueError("Src class is not handled, it is ")
    
    def sim_pwmat_fill(self, pwsrc:np.ndarray, pwdest:np.ndarray=None)->np.ndarray:
        """
        Fills in a pairwise comparison matrix with noisy votes based on pwsrc
        If pwsrc is None, we create a new matrix, otherwise we fill in pwdest
        with noisy values based on pwsrc and the self.pwvote_random parameter.
        In either case, we return the resulting noisy matrix
        """
        if pwdest is None:
            pwdest = deepcopy(pwsrc)
        size = len(pwsrc)
        for row in range(size):
            pwdest[row,row] = 1.0
            for col in range(row+1, size):
                val = pwsrc[row,col]
                if val >= 1:
                    nvote = self.pwvote_random(val)
                    pwdest[row, col]=nvote
                    pwdest[col, row]=1/nvote
                elif val!= 0:
                    nvote = self.pwvote_random(1/val)
                    pwdest[col, row] = nvote
                    pwdest[row, col] = 1/nvote
                else:
                    pwdest[row, col] = nvote
                    pwdest[col, row] = nvote
        return pwdest
    
    def sim_pwmat(self, pwsrc:np.ndarray, pwdest:np.ndarray=None)->np.ndarray:
        """
        creates a noisy pw comparison matrix from pwsrc, stores the matrix in pwdest (which
        is created if pwdest is None) calculates the resulting priority and returns that
        """
        pwdest = self.sim_pwmat_fill(pwsrc, pwdest)
        rval = self.pwprioritycalc(pwdest)
        return rval
    
    def sim_pw(self, pwsrc:Pairwise, pwdest:Pairwise)->np.ndarray:
        """
        Performs a simulation on a pairwise comparison matrix object and returns the
        resulting priorities
        """
        pwdest = self.sim_pw_fill(pwsrc, pwdest)
        mat = pwdest.matrix(self.username)
        rval = self.pwprioritycalc(mat)
        return rval
        
    
    def sim_pw_fill(self, pwsrc:Pairwise, pwdest:Pairwise=None)->Pairwise:
        """
        Fills in the pairwise comparison structure of pwdest with noisy pairwise data from pwsrc.
        If pwdest is None, we create one first, then fill in.  In either case, we return the pwdest
        object with new noisy data in it.
        """
        if pwdest is None:
            pwdest = deepcopy(pwsrc)
        for user in pwsrc.usernames():
            srcmat = pwsrc.matrix(user)
            destmat = pwdest.matrix(user)
            self.sim_pwmat_fill(srcmat, destmat)
        return pwdest
    
    def sim_direct_fill(self, directsrc:Direct, directdest:Direct=None)->Direct:
        """
        Fills in the direct data structure of directdest with noisy data from directsrc.
        If directdest is None, we create on as a deep copy of directsrc, then fill in.
        In either case, we return the directdest object with new noisy data in it.
        """
        if directdest is None:
            directdest = deepcopy(directsrc)
        for altpos in range(len(directdest)):
            orig = directsrc[altpos]
            newvote = self.directvote_random(orig)
            directdest.data[altpos] = newvote
        return directdest
        
    def sim_ahptree_fill(self, ahpsrc:AHPTree, ahpdest:AHPTree)->AHPTree:
        """
        Fills in the ahp tree structure of ahpdest with noisy data from ahpsrc.
        If ahpdest is None, we create one as a deepcopy of ahpsrc, then fill in.
        In either case, we return the ahpdest object with new noisy data in it.
        """
        pass

In [34]:
mc = MCAnp()

In [35]:
pw = np.array([
    [1, 1/2, 3],
    [2, 1, 5],
    [1/3, 1/5, 1]
])
rpw= mc.sim_pwmat_fill(pw)
rpw

array([[1.        , 0.37051142, 3.33842281],
       [2.69897217, 1.        , 5.80183993],
       [0.29954265, 0.17235912, 1.        ]])

In [36]:
[mc.sim_pwmat(pw) for i in range(20)]

[array([0.27096634, 0.6243696 , 0.10466406]),
 array([0.30053369, 0.6140516 , 0.0854147 ]),
 array([0.27060148, 0.61894665, 0.11045187]),
 array([0.30726993, 0.5940259 , 0.09870417]),
 array([0.35503492, 0.50633442, 0.13863066]),
 array([0.4268226 , 0.46644976, 0.10672764]),
 array([0.29151253, 0.59072901, 0.11775846]),
 array([0.35685006, 0.53412795, 0.10902199]),
 array([0.29312101, 0.60557346, 0.10130553]),
 array([0.29723375, 0.59655076, 0.10621549]),
 array([0.31288256, 0.58281427, 0.10430317]),
 array([0.29374678, 0.58335089, 0.12290234]),
 array([0.36008121, 0.52188087, 0.11803791]),
 array([0.27858529, 0.60907088, 0.11234383]),
 array([0.35254206, 0.53844782, 0.10901012]),
 array([0.31278362, 0.58495631, 0.10226007]),
 array([0.28857276, 0.60663839, 0.10478885]),
 array([0.35274742, 0.53920847, 0.1080441 ]),
 array([0.3353495 , 0.54748455, 0.11716595]),
 array([0.29354446, 0.55889581, 0.14755973])]

In [37]:
pwobj = Pairwise(alts=['alt '+str(i) for i in range(3)])
pwobj.vote_matrix(user_name='u1', val=pw)

## Checking that the deep copy is actually a deep copy
For some reason deepcopy was not copying the matrix, I had to overwrite
__deepcopy__ in Pairwise

In [38]:
pwobj.matrix('u1')

array([[1.        , 0.5       , 3.        ],
       [2.        , 1.        , 5.        ],
       [0.33333333, 0.2       , 1.        ]])

In [39]:
rpwobj = pwobj.__deepcopy__()

In [40]:
a=rpwobj
b=pwobj
a.df

,Name,Age,Matrix
u1,None,None,"[[1.0, 0.5, 3.0], [2.0, 1.0, 5.0], [0.33333333..."


In [41]:
display(a.df.loc['u1', 'Matrix']) 
display(b.df.loc['u1', 'Matrix'])

array([[1.        , 0.5       , 3.        ],
       [2.        , 1.        , 5.        ],
       [0.33333333, 0.2       , 1.        ]])

array([[1.        , 0.5       , 3.        ],
       [2.        , 1.        , 5.        ],
       [0.33333333, 0.2       , 1.        ]])

In [42]:
display(a.matrix('u1') is b.matrix('u1'))
display(a.matrix('u1') == b.matrix('u1'))

False

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

## Now let's try to simulate

In [43]:
[mc.sim_pw(pwobj, rpwobj) for i in range(20)]

[array([0.32769651, 0.57816026, 0.09414324]),
 array([0.25376675, 0.62931466, 0.11691859]),
 array([0.27883567, 0.58449588, 0.13666844]),
 array([0.45477241, 0.4270347 , 0.11819289]),
 array([0.26431129, 0.6355023 , 0.10018641]),
 array([0.29876401, 0.59962489, 0.10161109]),
 array([0.34550038, 0.55380804, 0.10069158]),
 array([0.30459242, 0.60605887, 0.08934872]),
 array([0.36884984, 0.53198076, 0.0991694 ]),
 array([0.29668052, 0.60784341, 0.09547606]),
 array([0.33241561, 0.55768566, 0.10989873]),
 array([0.34673935, 0.52480519, 0.12845546]),
 array([0.32134548, 0.57229042, 0.1063641 ]),
 array([0.24228032, 0.6293006 , 0.12841908]),
 array([0.35238749, 0.53553539, 0.11207712]),
 array([0.22048779, 0.65273267, 0.12677954]),
 array([0.35894768, 0.51985835, 0.12119397]),
 array([0.3002224, 0.5978758, 0.1019018]),
 array([0.29763988, 0.59458002, 0.10778011]),
 array([0.34731638, 0.54556302, 0.10712059])]

In [44]:
pwobj.matrix('u1')

array([[1.        , 0.5       , 3.        ],
       [2.        , 1.        , 5.        ],
       [0.33333333, 0.2       , 1.        ]])

## Try to simulate a direct data

In [45]:
dd = Direct(alt_names=['a1', 'a2', 'a3'])
dd.data[0]=0.5
dd.data[1]=0.3
dd.data[2]=0.2

In [51]:
rdd=mc.sim_direct_fill(dd)
rdd.data

a1    0.447397
a2    0.309603
a3    0.231653
dtype: float64